In [1]:
# Data stuff
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
import pandas as pd
import dask.dataframe as dd
import numpy as np

# Visual stuff
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# VIP STUFF!
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard

# Warnings
import warnings

# Feature engineering

In [2]:
# Read samples from data_gen folder
samples = dd.read_csv('../data_gen/samples/*.part',
                               dtype={'icd_code': 'object'})

# Read core admissions
core_admissions = dd.read_csv('../mimic-iv-0.4/core/admissions.csv',
                              usecols=['subject_id','hadm_id','admittime']
                             ).compute() # Move into memory(very small, no problem my friend)
# Read samples with lab
samples_with_lab = dd.read_csv('../data_gen/samples_with_lab/*.part', 
                               dtype={'value': 'object'}) # small enough my friend, but no

# Read samples with chartevents
samples_chart = dd.read_csv('../data_gen/samples_with_chart/*.part').compute()

# drop nan values from samples with lab
samples_with_lab = samples_with_lab.dropna(subset=['label']).compute()

# Lab measures to include, select top 35 avaiable and remove some
lab_value_list = samples_with_lab.label.value_counts().index[0:35].values.tolist()

# Ignore this warning
warnings.filterwarnings("ignore", 'This pattern has match groups')
# Speed-up by pre-computing tables
pre_gen_lab_tables = dict([(val, samples_with_lab[samples_with_lab.label.str.contains(val)].sort_values(by='charttime') ) for val in lab_value_list])

## Ensure each lab feature only has one unit type

In [4]:
# Check and handle multiple units, keep one with most frequent counts.
for item in list(pre_gen_lab_tables.keys()):
    unique_keys = pre_gen_lab_tables[item].valueuom.unique().tolist()
    if len(unique_keys) > 1:
        print(item, "units :", unique_keys)
        highest_oc_unit = pre_gen_lab_tables[item].valueuom.value_counts().index[0]
        print('Highest occurrence: ', pre_gen_lab_tables[item].valueuom.value_counts().index[0]) 
        print('\n')
        # Only keep highest occurrence unit
        pre_gen_lab_tables[item] = pre_gen_lab_tables[item][pre_gen_lab_tables[item].valueuom==highest_oc_unit]

Creatinine units : ['mg/dL', 'mg/mg', 'mg/g', 'Ratio', 'mg/24hr', 'mL/min']
Highest occurrence:  mg/dL


Hemoglobin units : ['g/dL', '%']
Highest occurrence:  g/dL


Phosphate units : ['mg/dL', nan, '/hpf']
Highest occurrence:  mg/dL


MCHC units : ['g/dL', '%']
Highest occurrence:  g/dL


MCH units : ['pg', 'g/dL', '%']
Highest occurrence:  pg


RDW units : ['fL', '%']
Highest occurrence:  %


pH units : ['units', nan]
Highest occurrence:  units


PT units : [nan, 'sec']
Highest occurrence:  sec


L units : ['%', 'K/uL', 'IU/L', nan, 'mmol/L', 'log10 IU/mL', 'mg/dL', 'log10 cop/mL', 'mg/L', 'Ratio', 'U/mL', '#/uL', 'IU/mL', 'U/L', 'U', 'log10 copies/mL', 'mIU/mL']
Highest occurrence:  IU/L


H units : ['units', 'g/dL', '%', 'pg', nan, '#/lpf', 'log10 IU/mL', 'uIU/mL', 'mg/dL', 'ng/mL', 'log10 cop/mL', 'Ratio', 'pg/mL', 'U/mL', 'IU/mL', '+/-', 'mIU/mL', 'U', 'log10 copies/mL', 'nmol/L', 'umol/L', 'mg/L', 'mmol/L', 'ug/dL']
Highest occurrence:  %


I units : ['%', nan, 'mg/dL', 'ug/dL',

## Functions

In [5]:
# Conversion numbers to get standard units
lbs_conv = 0.45359237 # to kg
inch_conv =  0.39370079 # to cm

# Given a hadm_id find BMI from samples with chart, height and weight is required
def BMI_find(hadm_id):
    p_data = samples_chart[samples_chart.hadm_id==hadm_id]
    h_inches =  p_data[p_data.label=='Height']
    h_cm =  p_data[p_data.label=='Height (cm)']
    w_kg =  p_data[p_data.label=='Admission Weight (Kg)']
    w_lbs = p_data[p_data.label=='Admission Weight (lbs.)']
    
    # Ensure that there is at least height and weight in standard units or convert from lollipops per unicorn(american)
    if ( (len(h_inches) > 0) | (len(h_cm) > 0) ) & ( (len(w_kg) > 0) | (len(w_lbs) > 0) ):
        if len(w_kg) > 0:
            weight = w_kg.value.values[0]
        else:
            weight = w_lbs.value.values[0] / lbs_conv 
        if len(h_cm) > 0:
            height = h_cm.value.values[0] / 100
        else: 
            height = (h_inches.value.values[0] / inch_conv) / 100
        
        BMI = float(weight / (height**2))
        # Return BMI, sanity check first, 11-98, others are clearly transcribed wrong(manually checked)
        if (BMI<98) & (BMI>11):
            return BMI
    # Failed to generate BMI
    return float('NaN')
    

# Return value for some lab feature for a hadm_id
def lab_label(label,hadm_id):
    temp = pre_gen_lab_tables.get(label)
    val = get_earliest_val(temp[temp.hadm_id==hadm_id])
    return val
   
# Get the earliest
def get_earliest_val(sub_df):
    if len(sub_df) < 1:
        return float('NaN')
    else:
        for item in sub_df[['value']].values:
            try:
                rtr = float(item)
                return rtr
            except:
                continue
        return float('NaN')
    

# Return times previously admitted
def times_prev_admitted(subject_id,hadm_id,admittime):
    return (core_admissions[
        (core_admissions.subject_id==subject_id) & 
        (core_admissions.hadm_id!=hadm_id) &
        (core_admissions.admittime < admittime) 
        ]).shape[0]
    
# Return array with new feature
def transform(df):
        return [
            df['hadm_id'],
            ((pd.Timestamp(df['dischtime'])-pd.Timestamp(df['admittime'])).days),
            float((df['anchor_age'] * 365 + (pd.Timestamp(df['admittime'])-pd.Timestamp(str(df['anchor_year']))).days)/365.25),
            times_prev_admitted(df.subject_id,df.hadm_id,df.admittime),
            BMI_find(df['hadm_id'])
        ] + [lab_label(val,df['hadm_id']) for val in lab_value_list]
        

## Apply feature engineering

In [6]:
# Run transform function to generate features
res = samples.apply(transform, axis=1, meta=(None, 'timedelta64[ns]'))
# Add to temporary dataframe
dataframe_temp = dd.from_pandas(pd.DataFrame(res.compute().to_list(),
                                             columns=['hadm_id',
                                                        'length_of_stay(days)', # days
                                                        'age_at_admission',
                                                        'times_prev_admitted',
                                                        'BMI'
                                                     ]+lab_value_list
                                            ), npartitions=samples.npartitions).compute()
# Merge temporary dataframe with samples(it already contains some features)
samples = dd.merge(samples,dataframe_temp,on=['hadm_id'],how='inner').compute()

# Filter dataset

In [7]:
# Include only some columns
feature_names = ['age_at_admission',
                 'gender','language',
                 'ethnicity',
                 'insurance',
                 'seq_num',
                 'marital_status',
                 'times_prev_admitted',
                 'length_of_stay(days)',
                 'BMI']+lab_value_list
samples = samples[feature_names+['hospital_expire_flag']] 

# Pre-process and write data set to data_gen

In [8]:
# Check unique values for all columns
print('Column, unique values in column:')
for i, col in enumerate(samples.columns):
    print(col, samples[col].nunique())

Column, unique values in column:
age_at_admission 13019
gender 2
language 2
ethnicity 8
insurance 3
seq_num 27
marital_status 4
times_prev_admitted 82
length_of_stay(days) 149
BMI 3418
Glucose 496
Potassium 165
Sodium 163
Chloride 141
Bicarbonate 49
Anion Gap 47
Creatinine 359
Urea Nitrogen 459
Magnesium 53
Hematocrit 420
Hemoglobin 152
Platelet Count 763
Phosphate 138
Calcium, Total 117
White Blood Cells 540
MCHC 118
MCH 260
MCV 78
Red Blood Cells 485
RDW 178
pH 113
PTT 828
PT 849
INR(PT) 0
Specimen Type 0
L 1046
H 469
I 165
RDW-SD 522
pO2 469
Calculated Total CO2 60
Base Excess 57
pCO2 123
Bilirubin, Total 249
Alanine Aminotransferase (ALT) 0
hospital_expire_flag 2


In [9]:
# Label encoding(binary)
print('Label encoded following columns(name):')
for column in samples.columns:
    unique_col_values = samples[column].nunique()
    if (unique_col_values == 2 and column!='hospital_expire_flag'):
        print(column)
        temp_new = le.fit_transform(samples[column].astype('category'))
        samples.drop(labels=[column], axis="columns", inplace=True)
        samples[column] = temp_new
        
# Rename to as only options are english or ? 
samples = samples.rename(columns={'language':'language_english'})

Label encoded following columns(name):
gender
language


In [10]:
# OneHotEncoding
one_hot_encode_cols = ['marital_status','insurance', 'ethnicity']

for i, col in enumerate(one_hot_encode_cols):
    print(pd.get_dummies(samples[col],prefix=col,dtype=int).columns)
    
# Prepare index join for axis 1
samples = samples.reset_index(drop=True)

# Encode
for i, col in enumerate(one_hot_encode_cols):
    temp = pd.get_dummies(samples[col],prefix=col,dtype=int)
    # drop unable to obtain and unknown (for any string)
    samples = pd.concat([temp,samples],axis=1)
    
# Drop columns which were encoded
samples = samples.drop(columns=one_hot_encode_cols)
    
samples.columns

Index(['marital_status_DIVORCED', 'marital_status_MARRIED',
       'marital_status_SINGLE', 'marital_status_WIDOWED'],
      dtype='object')
Index(['insurance_Medicaid', 'insurance_Medicare', 'insurance_Other'], dtype='object')
Index(['ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN',
       'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO',
       'ethnicity_OTHER', 'ethnicity_UNABLE TO OBTAIN', 'ethnicity_UNKNOWN',
       'ethnicity_WHITE'],
      dtype='object')


Index(['ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN',
       'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO',
       'ethnicity_OTHER', 'ethnicity_UNABLE TO OBTAIN', 'ethnicity_UNKNOWN',
       'ethnicity_WHITE', 'insurance_Medicaid', 'insurance_Medicare',
       'insurance_Other', 'marital_status_DIVORCED', 'marital_status_MARRIED',
       'marital_status_SINGLE', 'marital_status_WIDOWED', 'age_at_admission',
       'seq_num', 'times_prev_admitted', 'length_of_stay(days)', 'BMI',
       'Glucose', 'Potassium', 'Sodium', 'Chloride', 'Bicarbonate',
       'Anion Gap', 'Creatinine', 'Urea Nitrogen', 'Magnesium', 'Hematocrit',
       'Hemoglobin', 'Platelet Count', 'Phosphate', 'Calcium, Total',
       'White Blood Cells', 'MCHC', 'MCH', 'MCV', 'Red Blood Cells', 'RDW',
       'pH', 'PTT', 'PT', 'INR(PT)', 'Specimen Type', 'L', 'H', 'I', 'RDW-SD',
       'pO2', 'Calculated Total CO2', 'Base Excess', 'pCO2',
       'Bilirubin, Total', 'Alanine Aminotransferas

In [11]:
# Drop all columns containing these strings
strings_drop = ['unknown','?','unable to obtain','other']
cols_to_drop = []
for item in strings_drop:
    cols_to_drop = cols_to_drop +[col_name for col_name in 
     [col for col in samples.columns] 
     if col_name.lower().find(item.lower()) > -1]
samples = samples.drop(labels=cols_to_drop, axis=1)
print('dropped: ', cols_to_drop)

dropped:  ['ethnicity_UNKNOWN', 'ethnicity_UNABLE TO OBTAIN', 'ethnicity_OTHER', 'insurance_Other']


In [12]:
dd.from_pandas(samples, npartitions=32).to_csv('../data_gen/samples_transformed', index=False,)
print('done')

done
